# MultiQuery Retrieval 🤖

We are leveraging LangChains [MultiQueryRetriever](https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/) to rewrite the customers input and optimize it for RAG

## Import Libraries 🧑‍💻

We have leveraged all these libraries in the previous videos with the exception of the [MultiQueryRetriever](https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/) which will be used to rewrite the customers input.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import logging

## Establish Connection to GPT4o, Embeddings, and Azure Search

In [ ]:
model = AzureChatOpenAI(
    azure_deployment="gpt4o",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01"
)

embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment="embeddings",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

index_name: str = "products-optimized"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_SEARCH_KEY"),
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

## Set Logging Level 

This is set so we can see the querys the multiquery retriever generates.

In [ ]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

## Test a Question 🧪

In [ ]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=model
)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

retrieval_chain = (
    {"context": retriever_from_llm, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(retrieval_chain.invoke("what is the price of the home theater system?"))

## Determine Unique Documents Returned 📃

In [ ]:
unique_docs = retriever_from_llm.invoke("what is the price of the home theater system?")
len(unique_docs)